### Import library

In [1]:
import pandas as pd
import numpy as np

In [2]:
client_df = pd.read_csv('drug_client.csv', index_col='id')
product_df = pd.read_csv('drug_product.csv', index_col='id')
date_df = pd.read_csv('drug_date.csv', index_col='id')

### Change columns data type

In [3]:
date_df['expiration_date'] = pd.to_datetime(date_df['expiration_date'], dayfirst=True)
date_df['transaction_date'] = pd.to_datetime(date_df['transaction_date'], dayfirst=True)

### Add column

In [4]:
date_df['Date_difference'] = date_df['expiration_date'] - date_df['transaction_date']

### Cleansing and filtering date

In [5]:
date_df.Date_difference = (date_df.Date_difference / np.timedelta64(1,'D')).astype(int)

In [6]:
date_df = date_df[date_df['Date_difference'] > 0 ]

In [7]:
filt_client = client_df["transaction_completed"] == False
client_df = client_df.drop(index=client_df[filt_client].index)

### Rename columns

In [8]:
list_new_cols = [
    'Expiration_date',
    'Created_date',
    'Transaction_date'
]

for old_col, new_col in zip(date_df.columns, list_new_cols):
    date_df = date_df.rename(columns={old_col: new_col})

In [9]:
list_new_cols = [
    'First_name',
    'Last_name',
    'Email',
    'Gender',
    'Country',
    'City',
    'Card_type',
    'Quantity',
    'Transaction_completed'
]
for old_col, new_col in zip(client_df.columns, list_new_cols):
    client_df = client_df.rename(columns={old_col: new_col})

In [10]:
list_new_cols = [
    'Company',
    'Brand',
    'Name',
    'Price'
]
for old_col, new_col in zip(product_df.columns, list_new_cols):
    product_df = product_df.rename(columns={old_col: new_col})

### Concatenate Columns

In [11]:
client_df['Full_name'] = client_df['First_name'] + " "+ client_df['Last_name']

### Merge tables

In [12]:
df_merge = pd.merge(client_df, product_df, on='id')

In [13]:
df = pd.merge(df_merge, date_df, on='id')

In [14]:
df['Year']= df['Transaction_date'].dt.year
df['month'] = df['Transaction_date'].dt.month

### Add column

In [15]:
df['Price_'] = df['Price'] * df['Quantity']

In [16]:
mapping={
    1: 'January', 
    2: 'February', 
    3: 'March', 
    4: 'April', 
    5: 'May',
    6: 'June', 
    7: 'July',
    8: 'August', 
    9: 'September', 
    10: 'October', 
    11: 'November',
    12: 'December'
    
} 
df['Month']=df['month'].map(mapping)

In [17]:
df['Year'] = df['Year'].astype('str')
df['Date'] = df['Year'] + '-' +  df['Month']

In [18]:
columns = [
    'Company',
    'Name',
    'Full_name',
    'Date',
    'Quantity',
    'Price_'
]
df = df[columns]

### Groupby operation

In [19]:
data_grp = df.groupby(['Date'])
company_grp = df.groupby(['Company'])

In [20]:
price = company_grp["Price_"].sum()
date = company_grp["Date"].sum()

In [21]:
data_frame = pd.concat([price, date], axis="columns", sort=False)
data_frame

,Price_,Date
Company,,
3M ESPE Dental Products,159.04,2021-December
7-11,183.44,2021-October
7-Eleven,198.68,2022-January
ADIENNE SA,41.98,2021-April
"ALK-Abello, Inc.",188.00,2021-July2021-August
...,...,...
Zandi LLC,259.68,2021-June
"Zep, Inc",348.92,2022-February
Zydus Pharmaceuticals USA Inc,3.51,2021-June
